# ObjectDetectionToolsを使った学習

ここを実行する前に、Twitterで神のアカウントをフォローする事。
フォローしてないと、動かないよ多分

Twitter

> https://twitter.com/karaage0703?s=20


# ObjectDetectionToolsのクローン

ディレクトリ移動(連続でやってる場合は、既に同じディレクトリかも)

> $ cd /models/research

Git Clone

> $ git clone https://github.com/karaage0703/object_detection_tools


# アノテーションデータのLabelMapファイルのコピー

In [20]:
import shutil

BASE_PATH="vott/TF2OD-TFRecords-export/tf_label_map.pbtxt"
COPY_PATH="models/research/object_detection_tools/data/tf_label_map.pbtxt"

shutil.copyfile(BASE_PATH, COPY_PATH)

'models/research/object_detection_tools/data/tf_label_map.pbtxt'

# アノテーションデータのコピー

アノテーションデータはラーニングデータとテストデータに分割してコピーする必要があります。

そこで、下記の流れで行います。

- アノテーションデータのファイルリスト取得

- 7:3くらいに分割して、ラーニング用とテストデータディレクトリにコピーする


In [21]:
# ファイルリスト取得
import glob
FILE_LIST=glob.glob("vott/TF2OD-TFRecords-export/*")

# sklernでリストを分割
import numpy as np
from sklearn.model_selection import train_test_split
import os
# 30%テストでリスト分割
TRAIN_LIST, TEST_LIST = train_test_split(FILE_LIST,test_size=0.3)

# ラーニングデータコピー
for BASE_FILE_PATH in TRAIN_LIST:
    # ベースパスからファイル名だけ抜き出し
    FILE_NAME=os.path.basename(BASE_FILE_PATH)
    # ファイルコピー
    shutil.copyfile(BASE_FILE_PATH, "models/research/object_detection_tools/data/train/"+FILE_NAME)

# テストデータコピー
for BASE_FILE_PATH in TEST_LIST:
    # ベースパスからファイル名だけ抜き出し
    FILE_NAME=os.path.basename(BASE_FILE_PATH)
    # ファイルコピー
    shutil.copyfile(BASE_FILE_PATH, "models/research/object_detection_tools/data/val/"+FILE_NAME)

print("ファイルコピー完了")

ファイルコピー完了


# スクリプトで連番になるように修正

> $ cd object_detection_tools/data

> $ ./change_tfrecord_filename.sh

# Configファイルの修正

> ~/models/research/object_detection_tools/config/efficientdet_d0_coco17_tpu-32.config

下記のnum_classesの値をアノテーションしたラベル数に修正する

```
model {
  ssd {
    num_classes: 1
```

転移学習を行わないので、下記の項目を削除する

> fine_tune_checkpoint: "./object_detection_tools/models/efficientdet_d0_coco17_tpu-32/checkpoint/ckpt-0"


# ラーニング

> $ cd ~/models/research

> $ python object_detection/model_main_tf2.py --pipeline_config_path="./object_detection_tools/config/efficientdet_d0_coco17_tpu-32.config" --model_dir="./saved_model_01" --num_train_steps=10000 --alsologtostderr

# モデルディレクトリ

/workspaces/TF2_ObjectDetection/models/research/saved_model_01

# TensorBoardで確認
ディレクトリをコマンドで下記まで移動する

> /workspaces/TF2_ObjectDetection/models/research

TensorBoard起動

> tensorboard --logdir='saved_model_01'
